Main Break Sensitivity Analysis

Once the water main asset prediction/assessment function has been performed this script can be applied to review input sensitivity 
It is basically repeating a method to print results in the notebook using pandas data frame table summaries
ie query and display results using this technology

Each field of interest can be reviewed with a total count of breaks, total feet of pipe for the field value, then another table showing the value and the number of breaks per 1000ft of pipe having that value in order to gain understanding of the sensitivity of that value group to likelhood of break.

in the results we are looking at relatively high values for breaks per 1000 feet with shape length minimums having good statistically represetative groups

any factor we can think of and add, like hydrant flow static pressure, can be tested for the correlation with likelihood of break to itelligently adjust asset models and focus on scoring factors that statistically lead to higher likelihood of break

In [32]:
watermains = r"Water_Main_2021_12_3"

#make a selection for potable 

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)




id,value
0,a Layer object
1,6299


In [35]:
#what is the sensitivity of soil corrosive property to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['SoilCorrosionValue', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['SoilCorrosionValue']).sum()
print(table)
table = df.groupby(['SoilCorrosionValue']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#display(HTML(table.to_html()))

                    BreakCount  MilesOfPipe  Shape_Length
SoilCorrosionValue                                       
0                          118    80.023198  4.225225e+05
3                          148    94.801176  5.005502e+05
5                          375   256.582123  1.354754e+06
SoilCorrosionValue
0    0.279275
3    0.295675
5    0.276803
dtype: float64


overall there is weak correlation here between soil corrosivity and LOB.
medium corrosivity is more likely to break than high corrosivity soils.
if medium corrosivity is more likely to break than high corrisivity should we be scoring this smiliar weight to age or material?

In [37]:
#what is the sensitivity of operational pressure zone to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['OperationArea', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['OperationArea']).sum()
print(table)
table = df.groupby(['OperationArea']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



               BreakCount  MilesOfPipe  Shape_Length
OperationArea                                       
HP                    265   183.713349  9.700065e+05
LP                    376   247.693147  1.307820e+06
OperationArea
HP    0.273194
LP    0.287501
dtype: float64


Pipes break more frequently in the LP Central Service zone, but not a lot more

In [59]:
#the pressure zone doesn seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['HG_Score', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['HG_Score']).sum()
print(table)
table = df.groupby(['HG_Score']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 85-90 PSI
#we could explore the ranges a little more these ranges were based on a histogram I looked at


          BreakCount  MilesOfPipe  Shape_Length
HG_Score                                       
0                393   279.148539  1.473904e+06
1                107    84.399216  4.456279e+05
2                 30    23.250675  1.227636e+05
3                 38    15.495313  8.181525e+04
4                 41    20.899192  1.103477e+05
5                 32     8.213561  4.336760e+04
HG_Score
0    0.266639
1    0.240111
2    0.244372
3    0.464461
4    0.371553
5    0.737878
dtype: float64


In [61]:
#the pressure zone doesn seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['HG_Score', 'Material', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['HG_Score', 'Material']).sum()
print(table)
table = df.groupby(['HG_Score', 'Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 85-90 PSI
#we could explore the ranges a little more these ranges were based on a histogram I looked at


                   BreakCount  MilesOfPipe   Shape_Length
HG_Score Material                                        
0        CAS              194    43.575373  230077.968114
         COP               12     5.847888   30876.846711
         DIP               84    52.658676  278037.807404
         GP                 3     0.214813    1134.212754
         NIL                0     0.230002    1214.409874
         PE                 0     0.979943    5174.099954
         PVC               88   166.947945  881485.150819
         RCP                2     6.108684   32253.850049
         SP                 4     0.360996    1906.061471
         TTE                6     2.201439   11623.597532
         UNK                0     0.022780     120.280874
1        CAS               52    20.730680  109457.989289
         COP                0     1.110907    5865.588470
         DIP               22    12.243100   64643.566311
         GP                 0     0.002269      11.981977
         NIL  

In [62]:
#the pressure zone doesn seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = [ 'Material', 'HG_Score','BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material','HG_Score']).sum()
print(table)
table = df.groupby(['Material','HG_Score']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 85-90 PSI
#we could explore the ranges a little more these ranges were based on a histogram I looked at


                   BreakCount  MilesOfPipe   Shape_Length
Material HG_Score                                        
CAS      0                194    43.575373  230077.968114
         1                 52    20.730680  109457.989289
         2                  9     1.569787    8288.476264
         3                 12     2.109123   11136.169559
         4                 20     3.103239   16385.103938
         5                 17     1.714747    9053.865485
COP      0                 12     5.847888   30876.846711
         1                  0     1.110907    5865.588470
         2                  5     1.183534    6249.062020
         3                  2     0.876778    4629.387235
         4                  3     0.681154    3596.494806
         5                  1     0.348750    1841.399589
DIP      0                 84    52.658676  278037.807404
         1                 22    12.243100   64643.566311
         2                  8     5.855073   30914.786368
         3    

In [39]:
#what is the sensitivity of operational pressure zone and install era to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['OperationArea', 'Install_Era', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['OperationArea', 'Install_Era']).sum()
print(table)
table = df.groupby(['OperationArea', 'Install_Era']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#pipes installed in the last decade seem highly likely to break, but there arent that many of them in this run for some reason
#pipes from the 60's and 70's are more likely to break
# pipes from the '30s in central service zone are likely to break, but not as much in the west hills
# pipes from the 1900-1909 in central service zone

                           BreakCount  MilesOfPipe   Shape_Length
OperationArea Install_Era                                        
HP            1850-1899             0     0.002911      15.371373
              1920-1929             1     0.501112    2645.870235
              1930-1939             1     0.295203    1558.674288
              1940-1949             0     0.183842     970.684927
              1950-1959            26     6.539497   34528.543394
              1960-1969            46    14.123230   74570.653104
              1970-1979            78    18.729310   98890.756346
              1980-1989            46    26.680671  140873.945143
              1990-1999            47    40.912863  216019.918484
              2000-2009            16    44.094583  232819.395913
              2010-2019             4    30.697364  162082.084537
              2020-2029             0     0.952763    5030.586325
LP            1850-1899             2     0.373751    1973.407082
          

In [40]:
#what is the sensitivity of pipe material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Material', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material']).sum()
print(table)
table = df.groupby(['Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#Ductile iron is the best performing material
#PE is the worst
#PVC is a lot more likely to break than CI, DIP, GP, RCP, TTE

#display(HTML(table.to_html()))

          BreakCount  MilesOfPipe  Shape_Length
Material                                       
CAS              304    72.802949  3.843996e+05
COP               23    10.049011  5.305878e+04
DIP              145    81.152504  4.284852e+05
GP                 3     0.217471  1.148247e+03
NIL                0     0.690357  3.645084e+03
PE                 0     1.495773  7.897679e+03
PVC              148   250.839453  1.324432e+06
RCP                5    10.518401  5.553715e+04
SP                 4     0.360996  1.906061e+03
TTE                9     3.256801  1.719591e+04
UNK                0     0.022780  1.202809e+02
Material
CAS    0.790844
COP    0.433482
DIP    0.338401
GP     2.612678
NIL    0.000000
PE     0.000000
PVC    0.111746
RCP    0.090030
SP     2.098568
TTE    0.523380
UNK    0.000000
dtype: float64


In [41]:
#what is the sensitivity of pipe diameter to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Diameter', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Diameter']).sum()
print(table)
table = df.groupby(['Diameter']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#small pipes, 1.5 in and 2 in are high LOB
#6 in pipes are high LOB
#20 in pipes are high lob

#display(HTML(table.to_html()))

          BreakCount  MilesOfPipe  Shape_Length
Diameter                                       
0.75               0     0.045281  2.390838e+02
1.00               2     0.782129  4.129641e+03
1.50               1     0.053295  2.813993e+02
2.00              32    16.737629  8.837468e+04
3.00               1     0.787698  4.159048e+03
4.00              25     6.175200  3.260506e+04
6.00             208    56.904879  3.004578e+05
8.00             273   234.535262  1.238346e+06
10.00              7     1.917197  1.012280e+04
12.00             70    83.530314  4.410401e+05
14.00              0     1.038166  5.481514e+03
16.00              7    17.880549  9.440930e+04
20.00              6     1.038535  5.483464e+03
24.00              8     8.601581  4.541635e+04
30.00              0     0.077706  4.102897e+02
36.00              1     1.301074  6.869668e+03
Diameter
0.75     0.000000
1.00     0.484304
1.50     3.553669
2.00     0.362095
3.00     0.240440
4.00     0.766752
6.00     0.692277
8

In [42]:
#what is the sensitivity of age to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Install_Era', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Install_Era']).sum()
print(table)
table = df.groupby(['Install_Era']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

             BreakCount  MilesOfPipe   Shape_Length
Install_Era                                        
1850-1899             2     0.376663    1988.778455
1900-1909             1     0.069673     367.871921
1910-1919             1     1.166661    6159.971803
1920-1929            12     2.489592   13145.043587
1930-1939            13     1.085374    5730.777114
1940-1949             7     1.985954   10485.836646
1950-1959            75    19.719329  104118.056004
1960-1969           125    37.268769  196779.099350
1970-1979           159    51.826631  273644.611755
1980-1989            95    60.576203  319842.353083
1990-1999            89    96.208191  507979.247518
2000-2009            42    84.884395  448189.603542
2010-2019            20    71.105828  375438.770590
2020-2029             0     2.643235   13956.279591
Install_Era
1850-1899    1.005642
1900-1909    2.718337
1910-1919    0.162338
1920-1929    0.912892
1930-1939    2.268453
1940-1949    0.667567
1950-1959    0.720336
19